# Day 10: Big Context ≠ Better Memory

**Long-running agent sessions face two enemies: latency and "lost in the middle" syndrome. The ADK solves this with Context Caching and Context Compaction.**

## 课程来源
- [Advent of Agents 2025 - Day 10](https://adventofagents.com/)
- [ADK Context Compaction](https://google.github.io/adk-docs/context/compaction/)
- [ADK Context Caching](https://google.github.io/adk-docs/context/caching/)

## Learning Goals

1. 理解为什么大上下文不等于更好的记忆
2. 学习 Context Compaction：如何压缩历史防止 context rot
3. 学习 Context Caching：如何缓存重指令提升性能
4. 实践 ADK 中这两种技术的实现方式

---
## 1. 问题：Big Context ≠ Better Memory

### 两个敌人

| 敌人 | 问题 | 后果 |
|------|------|------|
| **Latency (延迟)** | 上下文越长，处理时间越久 | 用户体验差，成本高 |
| **Lost in the Middle** | LLM 对长上下文中间部分容易遗忘 | 回答不准确，产生幻觉 |

### 研究发现：Lost in the Middle 现象

```
Retrieval Accuracy by Position in Context:

100% ┤██                              ██
 90% ┤██                              ██
 80% ┤██                              ██
 70% ┤██ █                          █ ██
 60% ┤██ ██                        ██ ██
 50% ┤██ ██ █                    █ ██ ██
 40% ┤██ ██ ██ █              █ ██ ██ ██
 30% ┤██ ██ ██ ██ █        █ ██ ██ ██ ██
 20% ┤██ ██ ██ ██ ██ █  █ ██ ██ ██ ██ ██
 10% ┤██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██ ██
     └────────────────────────────────────
      Start                         End
           Position in Context
```

> LLM 对开头和结尾的信息记忆最好，中间部分容易被"遗忘"

---
## 2. ADK 的解决方案

ADK 提供两个互补的技术：

| 技术 | 目的 | 解决的问题 |
|------|------|------------|
| **Context Compaction** | 压缩/摘要历史对话 | Lost in the Middle, 延迟 |
| **Context Caching** | 缓存稳定的指令前缀 | 成本, 延迟 |

```
┌─────────────────────────────────────────────────────────────┐
│                    WORKING CONTEXT                          │
│  ┌─────────────────┬──────────────────────────────────┐    │
│  │ Stable Prefix   │ Variable Suffix                  │    │
│  │ (Context Cache) │ (Context Compaction)             │    │
│  │                 │                                  │    │
│  │ • System prompt │ • Compacted history summary      │    │
│  │ • Agent identity│ • Recent conversation turns      │    │
│  │ • Tools/Schema  │ • Current user message           │    │
│  └─────────────────┴──────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────┘
```

---
## 3. Setup

In [ ]:
# 安装依赖
# !uv pip install google-adk google-genai python-dotenv

import sys
from pathlib import Path

# Add project root to path
sys.path.insert(0, str(Path.cwd().parent))

from shared import get_api_key

# 加载 API key
GOOGLE_API_KEY = get_api_key()
print("✅ API Key loaded")

---
## 4. Context Compaction（上下文压缩）

### 什么是 Context Compaction？

当对话历史变得太长时，自动将旧的对话压缩成摘要，保持上下文在可控范围内。

### 为什么需要？

```python
# ❌ Without Compaction: Context grows unboundedly
Turn 1:   [System] + [User1] + [Agent1]                    = 1000 tokens
Turn 10:  [System] + [User1..10] + [Agent1..10]            = 10000 tokens
Turn 100: [System] + [User1..100] + [Agent1..100]          = 100000 tokens 💥

# ✅ With Compaction: Context stays bounded
Turn 1:   [System] + [User1] + [Agent1]                    = 1000 tokens
Turn 10:  [System] + [Summary1-5] + [User6..10] + [Agent6..10] = 3000 tokens
Turn 100: [System] + [Summary1-95] + [User96..100] + [Agent96..100] = 3000 tokens ✅
```

### Compaction 工作原理

```
Before Compaction:
┌──────────────────────────────────────────────────────────────┐
│ [System] [T1] [T2] [T3] [T4] [T5] [T6] [T7] [T8] [T9] [T10] │
└──────────────────────────────────────────────────────────────┘
                           ↓
                     LLM Summarization
                           ↓
After Compaction:
┌────────────────────────────────────────────────┐
│ [System] [Summary T1-T7] [T8] [T9] [T10]       │
└────────────────────────────────────────────────┘
```

### 关键概念

| 概念 | 说明 |
|------|------|
| **Compaction Threshold** | 触发压缩的 token 阈值 |
| **Recent Turns to Keep** | 保留最近 N 轮不压缩 |
| **Summary Model** | 用于生成摘要的模型 |

In [ ]:
# 模拟 Context Compaction 的概念

from typing import List, Dict

class ConversationTurn:
    """单轮对话"""
    def __init__(self, user: str, agent: str, tokens: int = 100):
        self.user = user
        self.agent = agent
        self.tokens = tokens
    
    def __repr__(self):
        return f"Turn(U: {self.user[:20]}... A: {self.agent[:20]}...)"


class MockCompactor:
    """模拟 ADK 的 Context Compaction 逻辑"""
    
    def __init__(
        self, 
        max_tokens: int = 4000,
        recent_turns_to_keep: int = 3,
        system_tokens: int = 500
    ):
        self.max_tokens = max_tokens
        self.recent_turns_to_keep = recent_turns_to_keep
        self.system_tokens = system_tokens
        self.summary = None
        self.summary_tokens = 0
    
    def calculate_tokens(self, turns: List[ConversationTurn]) -> int:
        """计算当前上下文大小"""
        turn_tokens = sum(t.tokens for t in turns)
        return self.system_tokens + self.summary_tokens + turn_tokens
    
    def needs_compaction(self, turns: List[ConversationTurn]) -> bool:
        """检查是否需要压缩"""
        return self.calculate_tokens(turns) > self.max_tokens
    
    def compact(self, turns: List[ConversationTurn]) -> tuple:
        """执行压缩，返回 (summary, remaining_turns)"""
        if not self.needs_compaction(turns):
            return self.summary, turns
        
        # 保留最近 N 轮
        keep_turns = turns[-self.recent_turns_to_keep:]
        compact_turns = turns[:-self.recent_turns_to_keep]
        
        # 生成摘要（模拟 LLM 调用）
        old_summary = self.summary or ""
        new_content = "\n".join(
            f"- User asked about {t.user[:30]}, Agent responded about {t.agent[:30]}"
            for t in compact_turns
        )
        
        self.summary = f"""Previous conversation summary:
{old_summary}
{new_content}""".strip()
        
        # 摘要通常比原内容小很多
        compacted_tokens = sum(t.tokens for t in compact_turns)
        self.summary_tokens = int(compacted_tokens * 0.2)  # ~20% of original
        
        print(f"📦 Compacted {len(compact_turns)} turns ({compacted_tokens} tokens) -> Summary ({self.summary_tokens} tokens)")
        
        return self.summary, keep_turns

print("✅ MockCompactor class defined")

In [ ]:
# 演示 Compaction 过程

compactor = MockCompactor(
    max_tokens=2000,      # 上下文限制 2000 tokens
    recent_turns_to_keep=3,  # 保留最近 3 轮
    system_tokens=500     # System prompt 500 tokens
)

# 模拟对话积累
conversation: List[ConversationTurn] = []

sample_turns = [
    ("What products do you have?", "We have electronics, clothing, and home goods."),
    ("Show me laptops under $1000", "Here are 5 laptops: MacBook Air, Dell XPS..."),
    ("Tell me more about the MacBook", "The MacBook Air M3 has 8GB RAM, 256GB SSD..."),
    ("What's the warranty?", "Apple offers 1-year limited warranty..."),
    ("Can I get extended warranty?", "Yes, AppleCare+ is available for $149..."),
    ("Add MacBook to cart", "Added MacBook Air M3 to your cart. Total: $999"),
    ("What payment methods?", "We accept credit cards, PayPal, Apple Pay..."),
    ("Do you have student discounts?", "Yes! Students get 10% off with valid ID."),
    ("Apply student discount", "Student discount applied! New total: $899.10"),
    ("Proceed to checkout", "Redirecting to secure checkout..."),
]

print("Simulating 10-turn conversation...\n")
print("=" * 60)

for i, (user_msg, agent_msg) in enumerate(sample_turns, 1):
    turn = ConversationTurn(user_msg, agent_msg, tokens=150)
    conversation.append(turn)
    
    current_tokens = compactor.calculate_tokens(conversation)
    print(f"\nTurn {i}: {current_tokens} tokens")
    
    if compactor.needs_compaction(conversation):
        print(f"⚠️  Exceeded {compactor.max_tokens} tokens, triggering compaction...")
        summary, conversation = compactor.compact(conversation)
        print(f"✅ After compaction: {compactor.calculate_tokens(conversation)} tokens")

In [ ]:
# 查看最终状态
print("Final Context Structure:")
print("=" * 60)
print(f"\n[System Prompt]: {compactor.system_tokens} tokens")
print(f"\n[Summary]: {compactor.summary_tokens} tokens")
if compactor.summary:
    print(compactor.summary)
print(f"\n[Recent Turns]: {len(conversation)} turns")
for i, turn in enumerate(conversation):
    print(f"  {i+1}. U: {turn.user[:40]}...")
    print(f"     A: {turn.agent[:40]}...")
print(f"\n📊 Total: {compactor.calculate_tokens(conversation)} tokens (limit: {compactor.max_tokens})")

---
## 5. ADK 中的 Context Compaction

ADK 提供内置的 Compaction 支持：

In [ ]:
from google.adk.agents import Agent
from google.adk.agents.run_config import RunConfig

# 创建带有 Compaction 配置的 Agent
agent_with_compaction = Agent(
    name="support_agent",
    model="gemini-2.0-flash",
    instruction="You are a helpful customer support agent.",
)

# RunConfig 中配置 Compaction
run_config = RunConfig(
    # 当 session 事件达到此数量时触发压缩
    max_llm_calls=50,  # 最大 LLM 调用次数
)

print("✅ Agent with compaction config created")
print(f"   Max LLM calls: {run_config.max_llm_calls}")

### Compaction 配置选项

```python
# ADK 的压缩配置（概念示例）
compaction_config = {
    "enabled": True,
    "threshold_tokens": 8000,      # 触发压缩的 token 阈值
    "target_tokens": 4000,         # 压缩后的目标大小
    "recent_turns_to_keep": 5,     # 保留最近 N 轮不压缩
    "summarization_model": "gemini-2.0-flash",  # 用于生成摘要的模型
}
```

---
## 6. Context Caching（上下文缓存）

### 什么是 Context Caching？

将稳定不变的上下文前缀（如 System Prompt、工具定义）缓存在 API 端，避免每次请求重复处理。

### Static vs Turn Instructions 回顾（Day 8）

```
┌──────────────────────────────────────────┐
│  STABLE PREFIX (可缓存)                  │
│  ├─ System Prompt                        │
│  ├─ Agent Identity                       │
│  ├─ Tool Definitions                     │
│  └─ Few-shot Examples                    │
├──────────────────────────────────────────┤
│  VARIABLE SUFFIX (每次请求动态)          │
│  ├─ Turn Instructions                    │
│  ├─ Retrieved Memory                     │
│  ├─ Compacted History                    │
│  └─ Current User Message                 │
└──────────────────────────────────────────┘
```

### Caching 的收益

```
Without Caching:
Request 1: Process [System: 2000 tokens] + [User: 100 tokens] = 2100 tokens processed
Request 2: Process [System: 2000 tokens] + [User: 150 tokens] = 2150 tokens processed
Request 3: Process [System: 2000 tokens] + [User: 120 tokens] = 2120 tokens processed
Total: 6370 tokens processed

With Caching:
Request 1: Process [System: 2000 tokens] + [User: 100 tokens] = 2100 tokens (cache miss)
Request 2: Cache hit [System] + Process [User: 150 tokens]    = 150 tokens processed
Request 3: Cache hit [System] + Process [User: 120 tokens]    = 120 tokens processed
Total: 2370 tokens processed (~63% reduction!)
```

In [ ]:
# 模拟 Context Caching 的成本计算

class CachingCalculator:
    """计算 Context Caching 的成本收益"""
    
    def __init__(
        self,
        static_tokens: int,
        input_price_per_1k: float = 0.075,  # Gemini 2.0 Flash 价格
        cached_price_per_1k: float = 0.01875,  # 缓存命中价格 (通常是 1/4)
    ):
        self.static_tokens = static_tokens
        self.input_price = input_price_per_1k / 1000
        self.cached_price = cached_price_per_1k / 1000
    
    def calculate_savings(self, num_requests: int, avg_dynamic_tokens: int) -> dict:
        """计算 N 次请求的成本对比"""
        # 无缓存
        no_cache_tokens = num_requests * (self.static_tokens + avg_dynamic_tokens)
        no_cache_cost = no_cache_tokens * self.input_price
        
        # 有缓存（第一次 miss，后续 hit）
        first_request = (self.static_tokens + avg_dynamic_tokens) * self.input_price
        cached_requests = (num_requests - 1) * (
            self.static_tokens * self.cached_price +  # 缓存命中部分
            avg_dynamic_tokens * self.input_price      # 动态部分
        )
        with_cache_cost = first_request + cached_requests
        
        savings = no_cache_cost - with_cache_cost
        savings_pct = (savings / no_cache_cost) * 100 if no_cache_cost > 0 else 0
        
        return {
            "requests": num_requests,
            "no_cache_tokens": no_cache_tokens,
            "no_cache_cost": no_cache_cost,
            "with_cache_cost": with_cache_cost,
            "savings": savings,
            "savings_pct": savings_pct
        }

# 示例：2000 token 的 system prompt
calc = CachingCalculator(static_tokens=2000)

print("Context Caching Cost Analysis")
print("=" * 50)
print(f"Static Prefix: 2000 tokens")
print(f"Avg Dynamic: 500 tokens/request")
print()

for num_requests in [10, 100, 1000]:
    result = calc.calculate_savings(num_requests, avg_dynamic_tokens=500)
    print(f"\n{num_requests} requests:")
    print(f"  Without cache: ${result['no_cache_cost']:.4f}")
    print(f"  With cache:    ${result['with_cache_cost']:.4f}")
    print(f"  Savings:       ${result['savings']:.4f} ({result['savings_pct']:.1f}%)")

---
## 7. ADK 中启用 Context Caching

ADK 支持通过 Gemini API 的 Context Caching 功能：

In [ ]:
# ADK 中使用 Context Caching 的概念示例

# 大型 System Prompt（适合缓存）
LARGE_SYSTEM_PROMPT = """
You are an expert financial advisor AI assistant.

## Your Expertise
- Investment portfolio management
- Retirement planning
- Tax optimization strategies
- Risk assessment and management
- Market analysis and trends

## Communication Guidelines
1. Always provide balanced perspectives
2. Clearly state assumptions and limitations
3. Use data and evidence to support recommendations
4. Explain complex concepts in accessible terms
5. Never provide specific investment advice without disclaimers

## Response Format
Structure your responses as follows:
- **Summary**: Brief overview (1-2 sentences)
- **Analysis**: Detailed breakdown
- **Recommendations**: Actionable next steps
- **Risks**: Potential downsides to consider
- **Disclaimer**: Appropriate legal/financial disclaimers

## Regulatory Compliance
- Always remind users to consult licensed professionals
- Never guarantee specific returns
- Disclose that AI advice is educational, not professional
- Follow SEC and FINRA guidelines for financial communication

## Tool Usage
You have access to the following tools:
- calculate_compound_interest(principal, rate, years)
- get_stock_price(ticker)
- analyze_portfolio(holdings)
- calculate_tax_impact(income, deductions)

[... 更多详细说明 ...]
""" * 3  # 模拟更长的 prompt

print(f"System Prompt size: ~{len(LARGE_SYSTEM_PROMPT.split())} words")
print(f"Estimated tokens: ~{len(LARGE_SYSTEM_PROMPT) // 4} tokens")
print("\n✅ This large prompt is ideal for Context Caching!")

In [ ]:
from google.adk.agents import Agent

# 工具定义
def calculate_compound_interest(principal: float, rate: float, years: int) -> dict:
    """Calculate compound interest.
    
    Args:
        principal: Initial investment amount
        rate: Annual interest rate (as decimal, e.g., 0.07 for 7%)
        years: Number of years
    
    Returns:
        Dictionary with final amount and total interest
    """
    final_amount = principal * (1 + rate) ** years
    total_interest = final_amount - principal
    return {
        "principal": principal,
        "final_amount": round(final_amount, 2),
        "total_interest": round(total_interest, 2),
        "years": years,
        "rate": f"{rate * 100}%"
    }

def get_stock_price(ticker: str) -> dict:
    """Get current stock price (mock)."""
    # 模拟数据
    prices = {
        "AAPL": 195.50,
        "GOOGL": 175.25,
        "MSFT": 425.80,
        "AMZN": 185.30
    }
    price = prices.get(ticker.upper(), 100.00)
    return {"ticker": ticker.upper(), "price": price, "currency": "USD"}


# 创建 Agent（Static Instructions 会被缓存）
financial_advisor = Agent(
    name="financial_advisor",
    model="gemini-2.0-flash",
    instruction=LARGE_SYSTEM_PROMPT,  # 这个大 prompt 适合缓存
    tools=[calculate_compound_interest, get_stock_price],
)

print("✅ Financial Advisor Agent created")
print("   The large system prompt will benefit from API-level caching")

### Caching 最佳实践

| ✅ Do | ❌ Don't |
|-------|----------|
| 缓存大型、稳定的 system prompts | 缓存经常变化的内容 |
| 缓存工具定义和 schema | 缓存用户特定的数据 |
| 缓存 few-shot 示例 | 缓存时间敏感的信息 |
| 设置合理的 TTL (30-90分钟) | 缓存太小的内容 (<1000 tokens) |

---
## 8. 结合使用：Caching + Compaction

两种技术可以完美配合使用：

In [ ]:
# 完整的上下文管理策略

class ContextManager:
    """结合 Caching 和 Compaction 的上下文管理器"""
    
    def __init__(
        self,
        system_prompt: str,
        max_context_tokens: int = 8000,
        compaction_threshold: int = 6000,
        recent_turns_to_keep: int = 5
    ):
        self.system_prompt = system_prompt
        self.system_tokens = len(system_prompt) // 4  # 估算
        self.max_context = max_context_tokens
        self.compaction_threshold = compaction_threshold
        self.recent_turns = recent_turns_to_keep
        
        self.history_summary = None
        self.summary_tokens = 0
        self.conversation = []
        self.cache_hits = 0
        self.total_requests = 0
    
    def build_context(self, user_message: str, turn_instructions: str = "") -> dict:
        """构建最终发送给 LLM 的上下文"""
        self.total_requests += 1
        
        # 检查是否需要压缩
        current_tokens = self._estimate_tokens()
        if current_tokens > self.compaction_threshold:
            self._compact()
        
        # 构建上下文
        context = {
            "system": self.system_prompt,  # <- 会被 API 缓存
            "messages": []
        }
        
        # 添加历史摘要（如果有）
        if self.history_summary:
            context["messages"].append({
                "role": "user",
                "content": f"[Previous conversation summary]\n{self.history_summary}"
            })
        
        # 添加最近对话
        for turn in self.conversation:
            context["messages"].append({"role": "user", "content": turn["user"]})
            if "agent" in turn:
                context["messages"].append({"role": "assistant", "content": turn["agent"]})
        
        # 添加 turn instructions + 新消息
        if turn_instructions:
            context["messages"].append({
                "role": "user",
                "content": f"[Turn Instructions]\n{turn_instructions}"
            })
        
        context["messages"].append({"role": "user", "content": user_message})
        
        # 模拟缓存命中（除了第一次）
        if self.total_requests > 1:
            self.cache_hits += 1
        
        return context
    
    def add_turn(self, user: str, agent: str):
        """添加一轮对话"""
        self.conversation.append({"user": user, "agent": agent})
    
    def _estimate_tokens(self) -> int:
        """估算当前上下文大小"""
        conv_tokens = sum(len(str(t)) // 4 for t in self.conversation)
        return self.system_tokens + self.summary_tokens + conv_tokens
    
    def _compact(self):
        """执行压缩"""
        if len(self.conversation) <= self.recent_turns:
            return
        
        to_compact = self.conversation[:-self.recent_turns]
        self.conversation = self.conversation[-self.recent_turns:]
        
        # 生成摘要
        summary_parts = []
        if self.history_summary:
            summary_parts.append(self.history_summary)
        
        summary_parts.append("Recent topics discussed:")
        for turn in to_compact:
            summary_parts.append(f"- {turn['user'][:50]}...")
        
        self.history_summary = "\n".join(summary_parts)
        self.summary_tokens = len(self.history_summary) // 4
        
        print(f"📦 Compacted {len(to_compact)} turns into summary")
    
    def get_stats(self) -> dict:
        """获取统计信息"""
        return {
            "total_requests": self.total_requests,
            "cache_hits": self.cache_hits,
            "cache_hit_rate": f"{(self.cache_hits / max(1, self.total_requests)) * 100:.1f}%",
            "current_context_tokens": self._estimate_tokens(),
            "active_turns": len(self.conversation),
            "has_summary": self.history_summary is not None
        }

print("✅ ContextManager class defined")

In [ ]:
# 演示完整流程

# 使用较小的阈值便于演示
ctx_manager = ContextManager(
    system_prompt=LARGE_SYSTEM_PROMPT,
    max_context_tokens=4000,
    compaction_threshold=2000,
    recent_turns_to_keep=3
)

# 模拟多轮对话
conversations = [
    ("What's a good retirement savings strategy?", "For retirement, consider a diversified approach..."),
    ("How much should I save monthly?", "A common rule is 15% of your income..."),
    ("What about 401k vs IRA?", "Both are excellent options. 401k offers higher limits..."),
    ("Calculate compound interest for $10000 at 7% for 30 years", "Using the calculator: $76,122.55"),
    ("What's AAPL stock price?", "Apple (AAPL) is currently at $195.50"),
    ("Should I invest in index funds?", "Index funds are great for passive investors..."),
    ("What are the risks?", "Main risks include market volatility and..."),
    ("Summarize our discussion", "We covered retirement strategies, savings rates..."),
]

print("Simulating conversation with Caching + Compaction...\n")
print("=" * 60)

for i, (user_msg, agent_response) in enumerate(conversations, 1):
    print(f"\n--- Turn {i} ---")
    
    # 构建上下文（会触发缓存和压缩）
    context = ctx_manager.build_context(
        user_message=user_msg,
        turn_instructions=f"Current turn: {i}"
    )
    
    # 记录对话
    ctx_manager.add_turn(user_msg, agent_response)
    
    # 显示状态
    stats = ctx_manager.get_stats()
    print(f"User: {user_msg[:50]}...")
    print(f"Context: {stats['current_context_tokens']} tokens, Cache hits: {stats['cache_hits']}")

In [ ]:
# 最终统计
print("\n" + "=" * 60)
print("FINAL STATISTICS")
print("=" * 60)

stats = ctx_manager.get_stats()
for key, value in stats.items():
    print(f"  {key}: {value}")

print("\n📊 Benefits achieved:")
print(f"  ✅ Cache hit rate: {stats['cache_hit_rate']}")
print(f"  ✅ Context bounded at ~{stats['current_context_tokens']} tokens")
print(f"  ✅ Compaction: {stats['has_summary']}")
print(f"  ✅ Active turns kept: {stats['active_turns']}")

---
## 9. Key Takeaways

### Big Context ≠ Better Memory

| 问题 | 解决方案 |
|------|----------|
| Lost in the Middle | Context Compaction 保持上下文精简 |
| 延迟增加 | 两者结合减少处理量 |
| 成本增长 | Context Caching 复用计算 |

### Context Compaction

- **何时使用**: 长对话、多轮交互
- **工作方式**: 自动摘要旧对话，保留最近 N 轮
- **收益**: 防止上下文无限增长，保持响应质量

### Context Caching  

- **何时使用**: 大型、稳定的 system prompts
- **工作方式**: API 缓存前缀，后续请求复用
- **收益**: 降低成本 60%+，减少延迟

### 最佳实践

```python
# 1. 分离 Static 和 Dynamic 内容
static_prompt = "...large, stable content..."  # 可缓存
turn_instructions = f"...dynamic, per-request..."  # 动态生成

# 2. 设置合理的 Compaction 阈值
compaction_config = {
    "threshold": 8000,  # 触发点
    "target": 4000,     # 目标大小
    "keep_recent": 5    # 保留最近 N 轮
}

# 3. 监控指标
- Cache hit rate
- Average context size
- Compaction frequency
- Response quality over time
```

---
## 10. Resources

### 官方文档
- [ADK Context Compaction](https://google.github.io/adk-docs/context/compaction/) - 压缩历史防止 context rot
- [ADK Context Caching](https://google.github.io/adk-docs/context/caching/) - 缓存重指令提升性能
- [ADK Context Documentation](https://google.github.io/adk-docs/context/) - 上下文管理总览

### 相关阅读
- [Lost in the Middle: How Language Models Use Long Contexts](https://arxiv.org/abs/2307.03172) - 研究论文
- [Context Compaction Explained (Visual)](https://x.com/Saboo_Shubham_/status/1978286461607911617) - 可视化解释
- [ADK Context Caching & Compaction Infographic](https://adventofagents.com/aoa-day10-context-caching-compaction.png) - 信息图

### Day 8 回顾
- Static vs Turn Instructions 模式
- Session、Memory、Artifacts 架构
- Context as a Compiled View 设计理念